# LangGraph and LangSmith - Agentic RAG Powered by LangChain

In the following notebook we'll complete the following tasks:

- 🤝 Breakout Room #1:
  1. Install required libraries
  2. Set Environment Variables
  3. Creating our Tool Belt
  4. Creating Our State
  5. Creating and Compiling A Graph!

  - 🤝 Breakout Room #2:
  1. Evaluating the LangGraph Application with LangSmith
  2. Adding Helpfulness Check and "Loop" Limits
  3. LangGraph for the "Patterns" of GenAI

# 🤝 Breakout Room #1

## Part 1: LangGraph - Building Cyclic Applications with LangChain

LangGraph is a tool that leverages LangChain Expression Language to build coordinated multi-actor and stateful applications that includes cyclic behaviour.

### Why Cycles?

In essence, we can think of a cycle in our graph as a more robust and customizable loop. It allows us to keep our application agent-forward while still giving the powerful functionality of traditional loops.

Due to the inclusion of cycles over loops, we can also compose rather complex flows through our graph in a much more readable and natural fashion. Effectively allowing us to recreate application flowcharts in code in an almost 1-to-1 fashion.

### Why LangGraph?

Beyond the agent-forward approach - we can easily compose and combine traditional "DAG" (directed acyclic graph) chains with powerful cyclic behaviour due to the tight integration with LCEL. This means it's a natural extension to LangChain's core offerings!

## Task 1:  Dependencies

We'll first install all our required libraries.

In [15]:
from IPython.display import display, Markdown

def pretty_print(message: str) -> str:
    display(Markdown(message))

In [1]:
!pip install -qU langchain langchain_openai langchain-community langgraph arxiv duckduckgo_search==5.3.1b1

## Task 2: Environment Variables

We'll want to set both our OpenAI API key and our LangSmith environment variables.

In [2]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

In [3]:
from uuid import uuid4

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"AIE4 - LangGraph - {uuid4().hex[0:8]}"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangSmith API Key: ")

## Task 3: Creating our Tool Belt

As is usually the case, we'll want to equip our agent with a toolbelt to help answer questions and add external knowledge.

There's a tonne of tools in the [LangChain Community Repo](https://github.com/langchain-ai/langchain/tree/master/libs/community/langchain_community/tools) but we'll stick to a couple just so we can observe the cyclic nature of LangGraph in action!

We'll leverage:

- [Duck Duck Go Web Search](https://github.com/langchain-ai/langchain/tree/master/libs/community/langchain_community/tools/ddg_search)
- [Arxiv](https://github.com/langchain-ai/langchain/tree/master/libs/community/langchain_community/tools/arxiv)

####🏗️ Activity #1:

Please add the tools to use into our toolbelt.

> NOTE: Each tool in our toolbelt should be a method.

In [4]:
from langchain_community.tools.ddg_search import DuckDuckGoSearchRun
from langchain_community.tools.arxiv.tool import ArxivQueryRun

tool_belt = [
    DuckDuckGoSearchRun(),
    ArxivQueryRun()
]

### Model

Now we can set-up our model! We'll leverage the familiar OpenAI model suite for this example - but it's not *necessary* to use with LangGraph. LangGraph supports all models - though you might not find success with smaller models - as such, they recommend you stick with:

- OpenAI's GPT-3.5 and GPT-4
- Anthropic's Claude
- Google's Gemini

> NOTE: Because we're leveraging the OpenAI function calling API - we'll need to use OpenAI *for this specific example* (or any other service that exposes an OpenAI-style function calling API.

In [5]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o", temperature=0)

Now that we have our model set-up, let's "put on the tool belt", which is to say: We'll bind our LangChain formatted tools to the model in an OpenAI function calling format.

In [6]:
model = model.bind_tools(tool_belt)

#### ❓ Question #1:

How does the model determine which tool to use?

**Answer**

Every tools comes with a description field, which is assigned at class declaration. If we use `@tool` decorator to convert a function into a tool then the docstring of the decorated function becomes the description of the tool. The LLM that the agent uses decides if and which tool to use on the basis of the prompt and the descriptions of the available tools.

## Task 4: Putting the State in Stateful

Earlier we used this phrasing:

`coordinated multi-actor and stateful applications`

So what does that "stateful" mean?

To put it simply - we want to have some kind of object which we can pass around our application that holds information about what the current situation (state) is. Since our system will be constructed of many parts moving in a coordinated fashion - we want to be able to ensure we have some commonly understood idea of that state.

LangGraph leverages a `StatefulGraph` which uses an `AgentState` object to pass information between the various nodes of the graph.

There are more options than what we'll see below - but this `AgentState` object is one that is stored in a `TypedDict` with the key `messages` and the value is a `Sequence` of `BaseMessages` that will be appended to whenever the state changes.

Let's think about a simple example to help understand exactly what this means (we'll simplify a great deal to try and clearly communicate what state is doing):

1. We initialize our state object:
  - `{"messages" : []}`
2. Our user submits a query to our application.
  - New State: `HumanMessage(#1)`
  - `{"messages" : [HumanMessage(#1)}`
3. We pass our state object to an Agent node which is able to read the current state. It will use the last `HumanMessage` as input. It gets some kind of output which it will add to the state.
  - New State: `AgentMessage(#1, additional_kwargs {"function_call" : "WebSearchTool"})`
  - `{"messages" : [HumanMessage(#1), AgentMessage(#1, ...)]}`
4. We pass our state object to a "conditional node" (more on this later) which reads the last state to determine if we need to use a tool - which it can determine properly because of our provided object!

In [7]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
import operator
from langchain_core.messages import BaseMessage

class AgentState(TypedDict):
  messages: Annotated[list, add_messages]

## Task 5: It's Graphing Time!

Now that we have state, and we have tools, and we have an LLM - we can finally start making our graph!

Let's take a second to refresh ourselves about what a graph is in this context.

Graphs, also called networks in some circles, are a collection of connected objects.

The objects in question are typically called nodes, or vertices, and the connections are called edges.

Let's look at a simple graph.

![image](https://i.imgur.com/2NFLnIc.png)

Here, we're using the coloured circles to represent the nodes and the yellow lines to represent the edges. In this case, we're looking at a fully connected graph - where each node is connected by an edge to each other node.

If we were to think about nodes in the context of LangGraph - we would think of a function, or an LCEL runnable.

If we were to think about edges in the context of LangGraph - we might think of them as "paths to take" or "where to pass our state object next".

Let's create some nodes and expand on our diagram.

> NOTE: Due to the tight integration with LCEL - we can comfortably create our nodes in an async fashion!

In [8]:
from langgraph.prebuilt import ToolNode

def call_model(state):
  messages = state["messages"]
  response = model.invoke(messages)
  return {"messages" : [response]}

tool_node = ToolNode(tool_belt)

Now we have two total nodes. We have:

- `call_model` is a node that will...well...call the model
- `tool_node` is a node which can call a tool

Let's start adding nodes! We'll update our diagram along the way to keep track of what this looks like!


In [9]:
from langgraph.graph import StateGraph, END

uncompiled_graph = StateGraph(AgentState)

uncompiled_graph.add_node("agent", call_model)
uncompiled_graph.add_node("action", tool_node)

Let's look at what we have so far:

![image](https://i.imgur.com/md7inqG.png)

Next, we'll add our entrypoint. All our entrypoint does is indicate which node is called first.

In [10]:
uncompiled_graph.set_entry_point("agent")

![image](https://i.imgur.com/wNixpJe.png)

Now we want to build a "conditional edge" which will use the output state of a node to determine which path to follow.

We can help conceptualize this by thinking of our conditional edge as a conditional in a flowchart!

Notice how our function simply checks if there is a "function_call" kwarg present.

Then we create an edge where the origin node is our agent node and our destination node is *either* the action node or the END (finish the graph).

It's important to highlight that the dictionary passed in as the third parameter (the mapping) should be created with the possible outputs of our conditional function in mind. In this case `should_continue` outputs either `"end"` or `"continue"` which are subsequently mapped to the action node or the END node.

In [11]:
def should_continue(state):
  last_message = state["messages"][-1]

  if last_message.tool_calls:
    return "action"

  return END

uncompiled_graph.add_conditional_edges(
    "agent",
    should_continue
)

Let's visualize what this looks like.

![image](https://i.imgur.com/8ZNwKI5.png)

Finally, we can add our last edge which will connect our action node to our agent node. This is because we *always* want our action node (which is used to call our tools) to return its output to our agent!

In [12]:
uncompiled_graph.add_edge("action", "agent")

Let's look at the final visualization.

![image](https://i.imgur.com/NWO7usO.png)

All that's left to do now is to compile our workflow - and we're off!

In [13]:
compiled_graph = uncompiled_graph.compile()

#### ❓ Question #2:

Is there any specific limit to how many times we can cycle?

If not, how could we impose a limit to the number of cycles?

**Answer**

There is no specific limit to the number of cycles. However we can specify this limit and check it in the `should_continue` function:

```python
MAX_ITERATIONS = 10 # or any other positive number

def should_continue(state):
  last_message = state["messages"][-1]

  num_iterations = _get_num_iterations(state)
      if num_iterations > MAX_ITERATIONS:
          return END

  if last_message.tool_calls:
    return "action"

  return 
```

## Using Our Graph

Now that we've created and compiled our graph - we can call it *just as we'd call any other* `Runnable`!

Let's try out a few examples to see how it fairs:

In [19]:
from langchain_core.messages import HumanMessage

inputs = {"messages" : [HumanMessage(content="Who is the current captain of the Winnipeg Jets?")]}

async for chunk in compiled_graph.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        # print(values["messages"])
        pretty_print(str(values["messages"]))
        # print("\n\n")

Receiving update from node: 'agent'


[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_szmfmCm8tLgVvv4TkHeCPYLH', 'function': {'arguments': '{"query":"current captain of the Winnipeg Jets 2023"}', 'name': 'duckduckgo_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 156, 'total_tokens': 181}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_157b3831f5', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-dc30bf09-5a7e-457e-81f8-358d8e8921d4-0', tool_calls=[{'name': 'duckduckgo_search', 'args': {'query': 'current captain of the Winnipeg Jets 2023'}, 'id': 'call_szmfmCm8tLgVvv4TkHeCPYLH', 'type': 'tool_call'}], usage_metadata={'input_tokens': 156, 'output_tokens': 25, 'total_tokens': 181})]

Receiving update from node: 'action'


[ToolMessage(content='September 12, 2023. There are not many honours in team sports bigger than being named captain. That honour was given to Winnipeg Jet forward Adam Lowry officially Tuesday morning as he becomes the ... - Sep 12, 2023 After a season without a captain, the Winnipeg Jets have named their new leader. Centre Adam Lowry, who is about to enter his 10th season as a Jet, was given the honour Tuesday. The Winnipeg Jets will have a captain for the 2023-24 season. After going captain-less in 2022-23, the Winnipeg Jets unveiled Adam Lowry as the club\'s new captain on Tuesday morning. "When I ... Posted September 12, 2023 9:29 am. Centre Adam Lowry was named the Winnipeg Jets new captain on Tuesday. Lowry is the third Jets captain since the team moved from Atlanta to Winnipeg in 2011. He follows Andrew Ladd and Blake Wheeler, who served as captain for five and six years respectively. The Winnipeg Jets have officially made the decision on which player will wear the captain\'s \'C\' for the 2023-24 season, and hopefully onward. That player is 30-year-old centre Adam Lowry.', name='duckduckgo_search', tool_call_id='call_szmfmCm8tLgVvv4TkHeCPYLH')]

Receiving update from node: 'agent'


[AIMessage(content='The current captain of the Winnipeg Jets, as of the 2023-24 season, is Adam Lowry.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 441, 'total_tokens': 465}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_157b3831f5', 'finish_reason': 'stop', 'logprobs': None}, id='run-c290390f-db7b-46d2-90b1-07121e070a66-0', usage_metadata={'input_tokens': 441, 'output_tokens': 24, 'total_tokens': 465})]

Let's look at what happened:

1. Our state object was populated with our request
2. The state object was passed into our entry point (agent node) and the agent node added an `AIMessage` to the state object and passed it along the conditional edge
3. The conditional edge received the state object, found the "tool_calls" `additional_kwarg`, and sent the state object to the action node
4. The action node added the response from the OpenAI function calling endpoint to the state object and passed it along the edge to the agent node
5. The agent node added a response to the state object and passed it along the conditional edge
6. The conditional edge received the state object, could not find the "tool_calls" `additional_kwarg` and passed the state object to END where we see it output in the cell above!

Now let's look at an example that shows a multiple tool usage - all with the same flow!

In [28]:
inputs = {"messages" : [HumanMessage(content="Search Arxiv for the QLoRA paper, then search each of the authors to find out their latest Tweet using DuckDuckGo.")]}

async for chunk in compiled_graph.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        pretty_print(f"Receiving update from node: **{node}**")
        if node == "action":
          pretty_print(f"Tool Used: **{values['messages'][0].name}**")
        # pretty_print(str(values["messages"]))
        for x in values["messages"]:
           pretty_print(f"The last message: {x.content}")
           pretty_print(f"Additional kwargs: {str(x.additional_kwargs)}")

        # print("\n\n")

Receiving update from node: **agent**

The last message: 

Additional kwargs: {'tool_calls': [{'id': 'call_1td48ug4qxkLWKp8xrqOd9eB', 'function': {'arguments': '{"query": "QLoRA"}', 'name': 'arxiv'}, 'type': 'function'}, {'id': 'call_S3miWIbHMFtCV4CIZyW0F9dz', 'function': {'arguments': '{"query": "QLoRA paper authors"}', 'name': 'duckduckgo_search'}, 'type': 'function'}], 'refusal': None}

Receiving update from node: **action**

Tool Used: **arxiv**

...

The last message: Luke Zettlemoyer is a research manager and site lead for FAIR Seattle. He is also a Professor in the Allen School of Computer Science & Engineering at the University of Washington. His research is in empirical computational semantics, where the goal is to build models that recover representations of the meaning of natural language text. Tags: miracle baby, luke shaffer, carilion, roanoke Twenty brain surgeries in two years - it's something most people couldn't imagine. But for two-year-old Luke Shaffer, it's his reality. Luke Shaffer, born at 23 weeks, has faced nearly 20 brain surgeries in his two years, with his family enduring countless challenges and showing remarkable resilience. Today we're joined by Luke Zettlemoyer, professor at University of Washington and a research manager at Meta. In our conversation with Luke, we cover multimodal generative AI, the effect of data on models, and the significance of open source and open science. We explore the grounding problem, the need for visual grounding and embodiment in ﻿ Twitter ﻿ Reddit. Join our list for notifications and early access to events ... About this Episode. Today we're joined by Luke Zettlemoyer, professor at University of Washington and a research manager at Meta. In our conversation with Luke, we cover multimodal generative AI, the effect of data on models, and the significance of open ...

Additional kwargs: {}

Receiving update from node: **agent**

The last message: Here are the latest updates and tweets from the authors of the QLoRA paper:

1. **Tim Dettmers**:
   - Tim Dettmers is joining Ai2 as an AI researcher. He specializes in efficient deep learning at the intersection of machine learning and NLP. He recently accepted the grand prize for QLoRA, a novel approach to finetuning pretrained models that significantly reduces the amount of GPU memory required.

2. **Artidoro Pagnoni**:
   - Artidoro Pagnoni's latest updates include his work on efficient finetuning of quantized LLMs. He has been involved in the development of QLoRA, which reduces memory usage enough to finetune a 65B parameter model on a single 48GB GPU.

3. **Ari Holtzman**:
   - Ari Holtzman, an alum and incoming professor at the University of Chicago, was recently involved in a heated on-air dispute on MSNBC. He is also part of the team behind QLoRA, which includes Allen School Ph.D. student Artidoro Pagnoni and professor Luke Zettlemoyer.

4. **Luke Zettlemoyer**:
   - Luke Zettlemoyer is a research manager and site lead for FAIR Seattle, as well as a Professor at the University of Washington. His research focuses on empirical computational semantics. He recently discussed multimodal generative AI, the effect of data on models, and the significance of open source and open science in a conversation.

These updates provide a glimpse into their recent activities and contributions to the field of AI and machine learning.

Additional kwargs: {'refusal': None}

####🏗️ Activity #2:

Please write out the steps the agent took to arrive at the correct answer:
1. Agent reads the prompt, decides to use the tools:
    1. arxiv tool to pull the QLoRA paper
    2. duckduckgo search for tweets of people the model assumes are authors of the QLoRA paper
2. Collecting outputs from tools:
    1. QLoRA paper and abstract from arxiv
    2. tweets found by duckduckgo
3. Agent reads the output from the tools

## Part 1: LangSmith Evaluator

### Pre-processing for LangSmith

To do a little bit more preprocessing, let's wrap our LangGraph agent in a simple chain.

In [20]:
def convert_inputs(input_object):
  return {"messages" : [HumanMessage(content=input_object["question"])]}

def parse_output(input_state):
  return input_state["messages"][-1].content

agent_chain = convert_inputs | compiled_graph | parse_output

In [21]:
agent_chain.invoke({"question" : "What is RAG?"})

'RAG stands for Retrieval-Augmented Generation. It is a technique used in natural language processing (NLP) and machine learning to improve the performance of language models by combining retrieval-based methods with generative models. Here’s a brief overview of how it works:\n\n1. **Retrieval**: In the first step, the system retrieves relevant documents or pieces of information from a large corpus or database. This is typically done using a retrieval model, such as BM25 or a dense retrieval model like DPR (Dense Passage Retrieval).\n\n2. **Augmentation**: The retrieved documents are then used to augment the input to the generative model. This means that the generative model has access to additional context or information that can help it produce more accurate and relevant responses.\n\n3. **Generation**: Finally, the generative model (often a transformer-based model like GPT-3) uses the augmented input to generate a response. The additional context provided by the retrieved documents 

### Task 1: Creating An Evaluation Dataset

Just as we saw last week, we'll want to create a dataset to test our Agent's ability to answer questions.

In order to do this - we'll want to provide some questions and some answers. Let's look at how we can create such a dataset below.

```python
questions = [
    "What optimizer is used in QLoRA?",
    "What data type was created in the QLoRA paper?",
    "What is a Retrieval Augmented Generation system?",
    "Who authored the QLoRA paper?",
    "What is the most popular deep learning framework?",
    "What significant improvements does the LoRA system make?"
]

answers = [
    {"must_mention" : ["paged", "optimizer"]},
    {"must_mention" : ["NF4", "NormalFloat"]},
    {"must_mention" : ["ground", "context"]},
    {"must_mention" : ["Tim", "Dettmers"]},
    {"must_mention" : ["PyTorch", "TensorFlow"]},
    {"must_mention" : ["reduce", "parameters"]},
]
```

####🏗️ Activity #3:

Please create a dataset in the above format with at least 5 questions.

In [22]:
questions = [
    "What optimizer is used in QLoRA?",
    "What data type was created in the QLoRA paper?",
    "What is a Retrieval Augmented Generation system?",
    "Who authored the QLoRA paper?",
    "What is the most popular deep learning framework?",
    "What significant improvements does the LoRA system make?"
]

answers = [
    {"must_mention" : ["paged", "optimizer"]},
    {"must_mention" : ["NF4", "NormalFloat"]},
    {"must_mention" : ["ground", "context"]},
    {"must_mention" : ["Tim", "Dettmers"]},
    {"must_mention" : ["PyTorch", "TensorFlow"]},
    {"must_mention" : ["reduce", "parameters"]},
]

Now we can add our dataset to our LangSmith project using the following code which we saw last Thursday!

In [23]:
from langsmith import Client

client = Client()
dataset_name = f"Retrieval Augmented Generation - Evaluation Dataset - {uuid4().hex[0:8]}"

dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Questions about the QLoRA Paper to Evaluate RAG over the same paper."
)

client.create_examples(
    inputs=[{"question" : q} for q in questions],
    outputs=answers,
    dataset_id=dataset.id,
)

#### ❓ Question #3:

How are the correct answers associated with the questions?

> NOTE: Feel free to indicate if this is problematic or not

In [25]:
from IPython.display import display, Markdown

def pretty_print(message: str) -> str:
    display(Markdown(message))

In [26]:
pretty_print(agent_chain.invoke({"question" : "How are the correct answers associated with the questions in LangSmith evaluation?"}))

In LangSmith evaluations, correct answers are typically associated with questions through a process that involves defining a set of expected answers or criteria that the responses should meet. Here’s a general outline of how this process might work:

1. **Question Definition**: Clearly define the questions that need to be evaluated. Each question should be specific and unambiguous.

2. **Expected Answer Specification**: For each question, specify the correct or expected answers. This can be done in several ways:
   - **Exact Match**: The answer must exactly match a predefined correct answer.
   - **Pattern Matching**: The answer must match a certain pattern or regular expression.
   - **Semantic Matching**: The answer must be semantically equivalent to the expected answer, even if the wording is different. This might involve using natural language processing techniques to determine equivalence.
   - **Range or Criteria**: For numerical or range-based answers, specify the acceptable range or criteria that the answer must meet.

3. **Evaluation Metrics**: Define the metrics that will be used to evaluate the answers. Common metrics include:
   - **Accuracy**: The percentage of correct answers.
   - **Precision and Recall**: For tasks involving multiple possible correct answers.
   - **F1 Score**: A balance between precision and recall.
   - **Human Judgment**: In some cases, human evaluators might be used to judge the correctness of answers based on predefined rubrics.

4. **Automated Evaluation**: Implement automated systems to compare the provided answers against the expected answers using the defined metrics. This might involve:
   - **String Comparison**: For exact matches.
   - **Regular Expressions**: For pattern matching.
   - **Semantic Analysis**: Using NLP techniques for semantic matching.
   - **Custom Logic**: For range or criteria-based evaluations.

5. **Human Review (if necessary)**: In cases where automated evaluation is challenging or ambiguous, human reviewers might be employed to make the final judgment.

6. **Feedback and Iteration**: Use the results of the evaluation to provide feedback and iterate on the question definitions, expected answers, and evaluation metrics to improve the accuracy and reliability of the evaluation process.

By following these steps, LangSmith evaluations can systematically associate correct answers with questions, ensuring a robust and reliable assessment of the responses.

### Task 2: Adding Evaluators

Now we can add a custom evaluator to see if our responses contain the expected information.

We'll be using a fairly naive exact-match process to determine if our response contains specific strings.

In [28]:
from langsmith.evaluation import EvaluationResult, run_evaluator

@run_evaluator
def must_mention(run, example) -> EvaluationResult:
    prediction = run.outputs.get("output") or ""
    required = example.outputs.get("must_mention") or []
    score = all(phrase in prediction for phrase in required)
    return EvaluationResult(key="must_mention", score=score)

#### ❓ Question #4:

What are some ways you could improve this metric as-is?

> NOTE: Alternatively you can suggest where gaps exist in this method.

Now that we have created our custom evaluator - let's initialize our `RunEvalConfig` with it!

In [30]:
from langchain.smith import RunEvalConfig, run_on_dataset

eval_config = RunEvalConfig(
    custom_evaluators=[must_mention],
)

Task 3: Evaluating

All that is left to do is evaluate our agent's response!

In [31]:
client.run_on_dataset(
    dataset_name=dataset_name,
    llm_or_chain_factory=agent_chain,
    evaluation=eval_config,
    verbose=True,
    project_name=f"RAG Pipeline - Evaluation - {uuid4().hex[0:8]}",
    project_metadata={"version": "1.0.0"},
)

View the evaluation results for project 'RAG Pipeline - Evaluation - 52a25363' at:
https://smith.langchain.com/o/36ad7c64-e702-5d92-ad6e-24634a4b396a/datasets/a44444c7-004e-4729-94e4-1349ce587127/compare?selectedSessions=534a95aa-cb84-4266-841b-ec6283240b3c

View all tests for Dataset Retrieval Augmented Generation - Evaluation Dataset - c444bab1 at:
https://smith.langchain.com/o/36ad7c64-e702-5d92-ad6e-24634a4b396a/datasets/a44444c7-004e-4729-94e4-1349ce587127
[------------------------------------------------->] 6/6

,feedback.must_mention,error,execution_time,run_id
count,6,0,6.000000,6
unique,2,0,NaN,6
top,True,NaN,NaN,49fa5c38-58e2-4b3c-9c83-69db96d28690
freq,4,NaN,NaN,1
mean,NaN,NaN,4.055137,NaN
std,NaN,NaN,0.804485,NaN
min,NaN,NaN,2.775695,NaN
25%,NaN,NaN,3.694220,NaN
50%,NaN,NaN,4.126236,NaN
75%,NaN,NaN,4.667058,NaN


{'project_name': 'RAG Pipeline - Evaluation - 52a25363',
 'results': {'818887fa-c4f5-462d-af5c-9975aa28ab48': {'input': {'question': 'What optimizer is used in QLoRA?'},
   'feedback': [EvaluationResult(key='must_mention', score=True, value=None, comment=None, correction=None, evaluator_info={}, feedback_config=None, source_run_id=UUID('c41f6ba1-137b-44a8-bc5f-d5c9272e8482'), target_run_id=None)],
   'execution_time': 4.145801,
   'run_id': '49fa5c38-58e2-4b3c-9c83-69db96d28690',
   'output': 'QLoRA (Quantized Low-Rank Adaptation) uses a combination of optimizers to manage memory efficiently during the fine-tuning of large language models. Specifically, it introduces "paged optimizers" to handle memory spikes. This approach allows QLoRA to backpropagate gradients through a frozen, 4-bit quantized pretrained language model into Low Rank Adapters (LoRA), significantly reducing memory usage without sacrificing performance.',
   'reference': {'must_mention': ['paged', 'optimizer']}},
  '19

## Part 2: LangGraph with Helpfulness:

### Task 3: Adding Helpfulness Check and "Loop" Limits

Now that we've done evaluation - let's see if we can add an extra step where we review the content we've generated to confirm if it fully answers the user's query!

We're going to make a few key adjustments to account for this:

1. We're going to add an artificial limit on how many "loops" the agent can go through - this will help us to avoid the potential situation where we never exit the loop.
2. We'll add to our existing conditional edge to obtain the behaviour we desire.

First, let's define our state again - we can check the length of the state object, so we don't need additional state for this.

In [32]:
class AgentState(TypedDict):
  messages: Annotated[list, add_messages]

Now we can set our graph up! This process will be almost entirely the same - with the inclusion of one additional node/conditional edge!

####🏗️ Activity #5:

Please write markdown for the following cells to explain what each is doing.

##### YOUR MARKDOWN HERE

In [33]:
graph_with_helpfulness_check = StateGraph(AgentState)

graph_with_helpfulness_check.add_node("agent", call_model)
graph_with_helpfulness_check.add_node("action", tool_node)

##### YOUR MARKDOWN HERE

In [34]:
graph_with_helpfulness_check.set_entry_point("agent")

##### YOUR MARKDOWN HERE

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

def tool_call_or_helpful(state):
  last_message = state["messages"][-1]

  if last_message.tool_calls:
    return "action"

  initial_query = state["messages"][0]
  final_response = state["messages"][-1]

  if len(state["messages"]) > 10:
    return "END"

  prompt_template = """\
  Given an initial query and a final response, determine if the final response is extremely helpful or not. Please indicate helpfulness with a 'Y' and unhelpfulness as an 'N'.

  Initial Query:
  {initial_query}

  Final Response:
  {final_response}"""

  prompt_template = PromptTemplate.from_template(prompt_template)

  helpfulness_check_model = ChatOpenAI(model="gpt-4")

  helpfulness_chain = prompt_template | helpfulness_check_model | StrOutputParser()

  helpfulness_response = helpfulness_chain.invoke({"initial_query" : initial_query.content, "final_response" : final_response.content})

  if "Y" in helpfulness_response:
    return "end"
  else:
    return "continue"

####🏗️ Activity #4:

Please write what is happening in our `tool_call_or_helpful` function!

##### YOUR MARKDOWN HERE

In [ ]:
graph_with_helpfulness_check.add_conditional_edges(
    "agent",
    tool_call_or_helpful,
    {
        "continue" : "agent",
        "action" : "action",
        "end" : END
    }
)

##### YOUR MARKDOWN HERE

In [ ]:
graph_with_helpfulness_check.add_edge("action", "agent")

##### YOUR MARKDOWN HERE

In [ ]:
agent_with_helpfulness_check = graph_with_helpfulness_check.compile()

##### YOUR MARKDOWN HERE

In [ ]:
inputs = {"messages" : [HumanMessage(content="Related to machine learning, what is LoRA? Also, who is Tim Dettmers? Also, what is Attention?")]}

async for chunk in agent_with_helpfulness_check.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        print(values["messages"])
        print("\n\n")

Receiving update from node: 'agent'
[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Mupqbkenc1akVPXQgIiiK4Lu', 'function': {'arguments': '{"query": "LoRA machine learning"}', 'name': 'duckduckgo_search'}, 'type': 'function'}, {'id': 'call_tnBAlnw05GC9gnhE5p2YxamG', 'function': {'arguments': '{"query": "Tim Dettmers"}', 'name': 'duckduckgo_search'}, 'type': 'function'}, {'id': 'call_v3fIuOcuNsGWIchiBh5Ey5Wy', 'function': {'arguments': '{"query": "Attention in machine learning"}', 'name': 'duckduckgo_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 76, 'prompt_tokens': 171, 'total_tokens': 247}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_157b3831f5', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-7dc7f0bd-8689-438d-a9d6-dee1f4440184-0', tool_calls=[{'name': 'duckduckgo_search', 'args': {'query': 'LoRA machine learning'}, 'id': 'call_Mupqbkenc1akVPXQgIiiK4Lu', 'type': 'tool_call'}, 

### Task 4: LangGraph for the "Patterns" of GenAI

Let's ask our system about the 4 patterns of Generative AI:

1. Prompt Engineering
2. RAG
3. Fine-tuning
4. Agents

In [ ]:
patterns = ["prompt engineering", "RAG", "fine-tuning", "LLM-based agents"]

In [ ]:
for pattern in patterns:
  what_is_string = f"What is {pattern} and when did it break onto the scene??"
  inputs = {"messages" : [HumanMessage(content=what_is_string)]}
  messages = agent_with_helpfulness_check.invoke(inputs)
  print(messages["messages"][-1].content)
  print("\n\n")

Prompt engineering is a concept primarily associated with the field of artificial intelligence, particularly in the context of natural language processing (NLP) and large language models like GPT-3. It involves the design and crafting of prompts (input text) to elicit desired responses from AI models. The goal is to optimize the input to get the most accurate, relevant, or useful output from the model.

### Key Aspects of Prompt Engineering:
1. **Crafting Effective Prompts**: Designing questions or statements that guide the AI to produce the desired output.
2. **Iterative Testing**: Continuously refining prompts based on the responses received to improve accuracy and relevance.
3. **Understanding Model Behavior**: Gaining insights into how the model interprets different types of input to better predict its responses.

### Emergence of Prompt Engineering:
Prompt engineering became more prominent with the advent of large-scale language models like OpenAI's GPT-3, which was released in Ju